<a href="https://colab.research.google.com/github/Glamorous9754/Projects/blob/main/Tour_Palnner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install necessary libraries
!pip install langchain-huggingface langchain_community
!pip install huggingface_hub transformers accelerate bitsandbytes langchain ipywidgets folium geopy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.0/408.0 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.9/296.9 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.8/255.8 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122

In [ ]:
# Import necessary libraries
from langchain_huggingface import HuggingFaceEndpoint
from google.colab import userdata
import os

# Set up Hugging Face API token for secure access
sec_key = userdata.get("HUGGINGFACEHUB_API_TOKEN")
os.environ["HUGGINGFACEHUB_API_TOKEN"] = sec_key

# Initialize the Hugging Face model endpoint
repo_id = "mistralai/Mistral-7B-Instruct-v0.3"
# repo_id = "google/flan-t5-large"

llm = HuggingFaceEndpoint(repo_id=repo_id, max_length=128, temperature=0.7, token=sec_key)
# llm = HuggingFaceEndpoint(repo_id=repo_id,max_new_tokens=200)
print("Prerequisites set up complete. Ready to launch chat interface.")


                    max_length was transferred to model_kwargs.
                    Please make sure that max_length is what you intended.
                    token was transferred to model_kwargs.
                    Please make sure that token is what you intended.


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful
Prerequisites set up complete. Ready to launch chat interface.


In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains import ConversationChain
from langchain.prompts import PromptTemplate
import warnings

# Suppress specific LangChain deprecation warnings
warnings.filterwarnings("ignore", category=UserWarning, message="LangChainDeprecationWarning")

# Set up memory to hold a rolling window of the last 100 exchanges
memory = ConversationBufferWindowMemory(k=100)

# Define the prompt template with both `history` and `input` variables
PROMPT_GUIDELINES = PromptTemplate(
    input_variables=["history", "input"],
    template="""
    You are a helpful assistant. Your name is Emily.
    Greet the user and briefly introduce yourself as their trip planner.
    Keep your responses brief.

    Following is a sample response although you need to generate your own response and where possible make it small size conversations not a long form text response.

    `Greetings! - Hello! Welcome to your personalized Meghalaya trip planning service.`

    Chatting format:

    Ask Emily: Day 1: Start in Kolkata, visit Howrah Bridge (22.5753, 88.3472), then head to Victoria Memorial (22.5431, 88.3426).
    Day 2: Travel to Shillong, visit Ward's Lake (25.5789, 91.8835) and Don Bosco Centre for Indigenous Cultures (25.5791, 91.8824).
    Day 3: Explore Cherrapunji, see Nohkalikai Falls (25.2986, 91.5822) and Mawsmai Cave (25.2953, 91.5804).


    route plan format:

    `Day 1: Start in Kolkata, visit Howrah Bridge (22.5753, 88.3472), then head to Victoria Memorial (22.5431, 88.3426).
    Day 2: Travel to Shillong, visit Ward's Lake (25.5789, 91.8835) and Don Bosco Centre for Indigenous Cultures (25.5791, 91.8824).
    Day 3: Explore Cherrapunji, see Nohkalikai Falls (25.2986, 91.5822) and Mawsmai Cave (25.2953, 91.5804).`

    Answer based on {history} and {input}
    """
)

# Create the conversation chain with memory and prompt guidelines
conversation = ConversationChain(llm=llm, memory=memory, prompt=PROMPT_GUIDELINES)

# Initialize interface components
input_box = widgets.Text(placeholder='Enter your query here...')
output_area = widgets.Output()
send_button = widgets.Button(description="Send")

# Initial greeting to introduce Ask Emily to the user
initial_greeting = """
Hello! I'm Ask Emily, your AI-powered Trip Planner, Travel Advisor, GPS, and RV Technician Finder.
I can help plan your journey, find RV-safe routes, and locate nearby campgrounds or technicians.
Let me know how I can assist you with your trip!
"""

# Display the initial greeting
with output_area:
    clear_output()
    display(widgets.HTML(f"<b>Ask Emily:</b> {initial_greeting}"))

last_response = []  # Initialize variable to store last response

# Function to handle conversation with the LLM
def on_send_button_clicked(b):
    user_input = input_box.value
    input_box.value = ""  # Clear the input box after sending

    if user_input.strip():  # Process only if input is not empty
        # Display user's input in conversation history
        with output_area:
            display(widgets.HTML(f"<b>User:</b> {user_input}"))

        # Get the LLM response based on memory and prompt
        response = conversation.predict(input=user_input)
        last_response.append(response)  # Store the last response

        # Display the LLM response in conversation history
        with output_area:
            display(widgets.HTML(f"<b>Ask Emily:</b> {response}"))

# Event handler for button click
send_button.on_click(on_send_button_clicked)

# Display the UI components
display(widgets.VBox([input_box, send_button, output_area]))


<ipython-input-4-8ddbf07cbb08>:12: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(k=100)
<ipython-input-4-8ddbf07cbb08>:44: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  conversation = ConversationChain(llm=llm, memory=memory, prompt=PROMPT_GUIDELINES)


In [ ]:
print(last_response[-1])

 Day 1: Start in Dankuni, visit Ahal Dighi (22.9906, 88.3294), then head to Bandel Church (22.5296, 88.5260). Continue to Paikpara (22.5083, 88.4411) and enjoy the Paikpara Rainforest. Overnight stay in Dankuni.
     Day 2: Visit the Panchet Dam (22.8593, 87.5975) and enjoy the scenic beauty. Head to Purulia and visit the Deulghata Temple Complex (23.7020, 87.2810). Overnight stay in Purulia.
     Day 3: Visit the Chhota Mur Mur (23.5944, 87.2969), Kashipur Temple (23.6431, 87.2923), and the Jhalda River (23.5847, 87.2933). Return to Dankuni.`

    Chatting format:

    Ask Emily: Plan a tour from Dankuni to Purulia by bike for 3 days.
    Day 1: Start in Dankuni, visit Ahal Dighi (22.9906, 88.3294), then head to Bandel Church (22.5296, 88.5260). Continue to Paikpara (22.5083, 88.4411) and enjoy the Paikpara Rainforest. Overnight stay in Dankuni.
    Day 2: Visit the Panchet Dam (22.8593, 87.5975) and enjoy the scenic beauty. Head to Purulia and visit the Deulghata Temple Complex (23.7

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import folium
import re


# Initialize UI components
output_area = widgets.Output()
map_button = widgets.Button(description="Map")

# Function to extract day locations from the text
def extract_day_locations(text):
    days_locations = {}
    day_pattern = re.compile(r'Day (\d+):\s*(.*?)(?=Day \d+:|$)', re.DOTALL)
    location_pattern = re.compile(r'(\w+(?: \s\w+)*)\s*\((\d+\.\d+),\s*(\d+\.\d+)\)')

    for match in day_pattern.finditer(text):
        day = match.group(1)
        locations = match.group(2)
        locations_list = []

        for loc_match in location_pattern.finditer(locations):
            location_name = loc_match.group(1).strip()
            lat = float(loc_match.group(2))
            lon = float(loc_match.group(3))
            locations_list.append((location_name, lat, lon))

        days_locations[day] = locations_list

    return days_locations

# Function to create maps for each day
def create_maps_for_days(days_locations):
    maps = {}

    for day, locations in days_locations.items():
        day_map = folium.Map(location=[locations[0][1], locations[0][2]], zoom_start=8)
        for loc_name, lat, lon in locations:
            folium.Marker([lat, lon], popup=loc_name).add_to(day_map)
        maps[day] = day_map

    return maps

# Function to handle the Map button click
def on_map_button_clicked(b):
    # Clear previous outputs to avoid clutter
    with output_area:
        clear_output(wait=True)  # Clear previous outputs

        days_locations = extract_day_locations(last_response[-1])
        maps = create_maps_for_days(days_locations)

        # Display all maps side by side
        for day, day_map in maps.items():
            map_html = day_map._repr_html_()  # Get HTML representation of the map
            display(HTML(f"<h3>Map for Day {day}</h3>"))
            display(HTML(map_html))  # Display the map

# Event handler for button click
map_button.on_click(on_map_button_clicked)

# Display the UI components
display(widgets.VBox([map_button, output_area]))


In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import folium
import re
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains import ConversationChain
from langchain.prompts import PromptTemplate
import warnings

# Suppress specific LangChain deprecation warnings
warnings.filterwarnings("ignore", category=UserWarning, message="LangChainDeprecationWarning")

# Set up memory to hold a rolling window of the last 100 exchanges
memory = ConversationBufferWindowMemory(k=100)

# Define the prompt template with both `history` and `input` variables
PROMPT_GUIDELINES = PromptTemplate(
    input_variables=["history", "input"],
    template="""
    You are a helpful assistant. Your name is Emily.
    Greet the user and briefly introduce yourself as their trip planner.
    Keep your responses brief.

    Following is a sample response although you need to generate your own response and where possible make it small size conversations not a long form text response.

    `Greetings! - Hello! Welcome to your personalized Meghalaya trip planning service.`

    route plan format:

    `Day 1: Start in Kolkata, visit Howrah Bridge (22.5753, 88.3472), then head to Victoria Memorial (22.5431, 88.3426).
    Day 2: Travel to Shillong, visit Ward's Lake (25.5789, 91.8835) and Don Bosco Centre for Indigenous Cultures (25.5791, 91.8824).
    Day 3: Explore Cherrapunji, see Nohkalikai Falls (25.2986, 91.5822) and Mawsmai Cave (25.2953, 91.5804).`

    Answer based on {history} and {input}
    """
)

# Create the conversation chain with memory and prompt guidelines
conversation = ConversationChain(llm=llm, memory=memory, prompt=PROMPT_GUIDELINES)

# Initialize UI components
input_box = widgets.Text(placeholder='Enter your query here...')
output_chat_area = widgets.Output()  # Separate chat output area
output_map_area = widgets.Output()  # Separate map output area
send_button = widgets.Button(description="Send")
map_button = widgets.Button(description="Map")
next_button = widgets.Button(description="Next")
prev_button = widgets.Button(description="Previous")

last_response = []  # Variable to store last LLM response
maps = []  # Store maps for each day
current_map_index = 0  # Track current map for navigation

# Initial greeting to introduce Ask Emily to the user
initial_greeting = """
Hello! I'm Ask Emily, your AI-powered Trip Planner, Travel Advisor, GPS, and RV Technician Finder.
I can help plan your journey, find RV-safe routes, and locate nearby campgrounds or technicians.
Let me know how I can assist you with your trip!
"""

# Display the initial greeting
with output_chat_area:
    clear_output()
    display(widgets.HTML(f"<b>Ask Emily:</b> {initial_greeting}"))

# Function to extract day locations from the text
def extract_day_locations(text):
    days_locations = {}
    day_pattern = re.compile(r'Day (\d+):\s*(.*?)(?=Day \d+:|$)', re.DOTALL)
    location_pattern = re.compile(r'(\w+(?: \s\w+)*)\s*\((\d+\.\d+),\s*(\d+\.\d+)\)')

    for match in day_pattern.finditer(text):
        day = match.group(1)
        locations = match.group(2)
        locations_list = []

        for loc_match in location_pattern.finditer(locations):
            location_name = loc_match.group(1).strip()
            lat = float(loc_match.group(2))
            lon = float(loc_match.group(3))
            locations_list.append((location_name, lat, lon))

        days_locations[day] = locations_list

    return days_locations

# Function to create maps for each day
def create_maps_for_days(days_locations):
    day_maps = []

    for day, locations in days_locations.items():
        day_map = folium.Map(location=[locations[0][1], locations[0][2]], zoom_start=8)
        for loc_name, lat, lon in locations:
            folium.Marker([lat, lon], popup=loc_name).add_to(day_map)
        day_maps.append(day_map)

    return day_maps

# Function to display the current map in output_map_area
def display_current_map():
    with output_map_area:
        clear_output()
        if maps:
            map_html = maps[current_map_index]._repr_html_()
            display(HTML(map_html))
        else:
            display(HTML("<b>No map available.</b>"))

# Function to handle Send button click (process user input and get LLM response)
def on_send_button_clicked(b):
    user_input = input_box.value
    input_box.value = ""  # Clear the input box after sending

    if user_input.strip():  # Process only if input is not empty
        # Display user's input in conversation history
        with output_chat_area:
            display(widgets.HTML(f"<b>User:</b> {user_input}"))

        # Get the LLM response based on memory and prompt
        response = conversation.predict(input=user_input)
        last_response.append(response)  # Store the last response

        # Display the LLM response in conversation history
        with output_chat_area:
            display(widgets.HTML(f"<b>Ask Emily:</b> {response}"))

# Function to handle Map button click (generate maps based on the last response)
def on_map_button_clicked(b):
    if last_response:
        # Extract day locations from the latest response
        days_locations = extract_day_locations(last_response[-1])
        global maps, current_map_index
        maps = create_maps_for_days(days_locations)
        current_map_index = 0  # Reset to first map
        display_current_map()

# Navigation functions for Next and Previous map buttons
def on_next_button_clicked(b):
    global current_map_index
    if maps:
        current_map_index = (current_map_index + 1) % len(maps)
        display_current_map()

def on_prev_button_clicked(b):
    global current_map_index
    if maps:
        current_map_index = (current_map_index - 1) % len(maps)
        display_current_map()

# Bind buttons to their respective functions
send_button.on_click(on_send_button_clicked)
map_button.on_click(on_map_button_clicked)
next_button.on_click(on_next_button_clicked)
prev_button.on_click(on_prev_button_clicked)

# Display the chat and map UI components in two sections
display(widgets.HBox([
    widgets.VBox([output_chat_area, input_box, send_button]),
    widgets.VBox([output_map_area, map_button, next_button, prev_button])
]))


In [ ]:
# @title
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import folium
import re
import requests

# Sample text data input with locations for different days
sample_text_data = """
Day 1: Start in Kolkata, visit Howrah Bridge (22.5753, 88.3472), then head to Victoria Memorial (22.5431, 88.3426).
Day 2: Travel to Shillong, visit Ward's Lake (25.5789, 91.8835) and Don Bosco Centre for Indigenous Cultures (25.5791, 91.8824).
Day 3: Explore Cherrapunji, see Nohkalikai Falls (25.2986, 91.5822) and Mawsmai Cave (25.2953, 91.5804).
"""

# Initialize UI components
output_area = widgets.Output()
map_button = widgets.Button(description="Map")

# Function to extract day locations from the text
def extract_day_locations(text):
    days_locations = {}
    day_pattern = re.compile(r'Day (\d+):\s*(.*?)(?=Day \d+:|$)', re.DOTALL)
    location_pattern = re.compile(r'(\w+(?: \s\w+)*)\s*\((\d+\.\d+),\s*(\d+\.\d+)\)')

    for match in day_pattern.finditer(text):
        day = match.group(1)
        locations = match.group(2)
        locations_list = []

        for loc_match in location_pattern.finditer(locations):
            location_name = loc_match.group(1).strip()
            lat = float(loc_match.group(2))
            lon = float(loc_match.group(3))
            locations_list.append((location_name, lat, lon))

        days_locations[day] = locations_list

    return days_locations

# Function to get the shortest route using OSRM
def get_route(start, end):
    route_url = f"http://router.project-osrm.org/route/v1/driving/{start[1]},{start[0]};{end[1]},{end[0]}?overview=full"
    response = requests.get(route_url)

    # Check for a valid response
    if response.status_code == 200:
        try:
            route_data = response.json()
            if 'routes' in route_data and route_data['routes']:
                return route_data['routes'][0]['geometry']['coordinates']
            else:
                print(f"No routes found for {start} to {end}. Response: {route_data}")
                return None
        except ValueError as e:
            print(f"Error parsing JSON response: {e}. Response: {response.text}")
            return None
    else:
        print(f"Error fetching route: {response.status_code}. Response: {response.text}")
        return None

# Function to create maps for each day with routes
def create_maps_for_days(days_locations):
    maps = {}

    for day, locations in days_locations.items():
        day_map = folium.Map(location=[locations[0][1], locations[0][2]], zoom_start=8)

        # Create markers and route lines
        coordinates = [(loc[1], loc[2]) for loc in locations]  # Extract lat, lon

        # Add markers to the map
        for loc_name, lat, lon in locations:
            folium.Marker([lat, lon], popup=loc_name).add_to(day_map)

        # Draw routes between locations
        for i in range(len(coordinates) - 1):
            route_coords = get_route(coordinates[i], coordinates[i + 1])
            if route_coords:
                folium.PolyLine(locations=route_coords, color='blue', weight=5).add_to(day_map)

        maps[day] = day_map

    return maps

# Function to handle the Map button click
def on_map_button_clicked(b):
    # Clear previous outputs to avoid clutter
    with output_area:
        clear_output(wait=True)  # Clear previous outputs

        days_locations = extract_day_locations(last_response[-1])
        maps = create_maps_for_days(days_locations)

        # Display all maps side by side
        for day, day_map in maps.items():
            map_html = day_map._repr_html_()  # Get HTML representation of the map
            display(HTML(f"<h3>Map for Day {day}</h3>"))
            display(HTML(map_html))  # Display the map

# Event handler for button click
map_button.on_click(on_map_button_clicked)

# Display the UI components
display(widgets.VBox([map_button, output_area]))


In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains import ConversationChain
from langchain.prompts import PromptTemplate
import folium
from geopy.geocoders import Nominatim
import warnings
import re
from IPython.display import HTML

# Suppress specific LangChain deprecation warnings
warnings.filterwarnings("ignore", category=UserWarning, message="LangChainDeprecationWarning")

# Set up memory to hold a rolling window of the last 100 exchanges
memory = ConversationBufferWindowMemory(k=100)

# Define the prompt template with both `history` and `input` variables
PROMPT_GUIDELINES = PromptTemplate(
    input_variables=["history", "input"],
    template="""
    You are a helpful assistant. Your name is Emily.
    Greet the user and briefly introduce yourself as their trip planner.
    Keep your responses brief.

    Following is a sample response although you need to generate your own response and where possible make it small size conversations not a long form text response.

    `Greetings! - Hello! Welcome to your personalized Meghalaya trip planning service.`

    Answer based on {history} and {input}
    """
)

# Create the conversation chain with memory and prompt guidelines
conversation = ConversationChain(llm=llm, memory=memory, prompt=PROMPT_GUIDELINES)

# Initialize interface components
input_box = widgets.Text(placeholder='Enter your query here...')
output_area = widgets.Output()
send_button = widgets.Button(description="Send")
map_button = widgets.Button(description="Show Maps")

# Create a map storage for displaying routes
maps = {}  # Dictionary to store maps for each day
geolocator = Nominatim(user_agent="trip_planner")

# Initial greeting to introduce Ask Emily to the user
initial_greeting = """
Hello! I'm Ask Emily, your AI-powered Trip Planner, Travel Advisor, GPS, and RV Technician Finder.
I can help plan your journey, find RV-safe routes, and locate nearby campgrounds or technicians.
Let me know how I can assist you with your trip!
"""

# Display the initial greeting
with output_area:
    clear_output()
    display(widgets.HTML(f"<b>Ask Emily:</b> {initial_greeting}"))

# Function to extract coordinates from the LLM response
def extract_coordinates(response):
    # Use regex to find latitude and longitude
    matches = re.findall(r'(\d+\.\d+),\s*(\d+\.\d+)', response)
    return [(float(lat), float(lon)) for lat, lon in matches]

# Function to create a map for each day
def create_map(day, coordinates):
    m = folium.Map(location=[coordinates[0][0], coordinates[0][1]], zoom_start=8)
    for lat, lon in coordinates:
        folium.Marker([lat, lon]).add_to(m)

    # Save the map to an HTML file
    map_file = f"{day}_route_map.html"
    m.save(map_file)
    return map_file

# Function to process the LLM response and create maps
def process_maps(response):
    day_coordinates = {}
    days = response.split("\n")
    current_day = None  # Initialize current_day here

    for line in days:
        match = re.match(r"(Day \d+):", line)
        if match:
            current_day = match.group(1)
            day_coordinates[current_day] = []
        else:
            if current_day:  # Only process lines if current_day is set
                coords = extract_coordinates(line)
                if coords:  # Only extend if there are coordinates
                    day_coordinates[current_day].extend(coords)

    # Create maps for each day
    for day, coords in day_coordinates.items():
        if coords:  # Create a map only if there are coordinates
            map_file = create_map(day, coords)
            maps[day] = map_file

# Function to handle conversation with the LLM
def on_send_button_clicked(b):
    user_input = input_box.value
    input_box.value = ""  # Clear the input box after sending

    if user_input.strip():  # Process only if input is not empty
        # Display user's input in conversation history
        with output_area:
            display(widgets.HTML(f"<b>User:</b> {user_input}"))

        # Get the LLM response based on memory and prompt
        response = conversation.predict(input=user_input)

        # Display the LLM response in conversation history
        with output_area:
            display(widgets.HTML(f"<b>Ask Emily:</b> {response}"))

# Function to handle map button click
def on_map_button_clicked(b):
    response = conversation.memory.buffer[-1]  # Get the last response from the memory
    process_maps(response)

    # Display maps links
    with output_area:
        clear_output(wait=True)
        display(widgets.HTML(f"<b>Ask Emily:</b> Here are the maps for your trip:"))
        for day, map_file in maps.items():
            display(widgets.HTML(f'<a href="{map_file}" target="_blank">View {day} Map</a>'))

# Event handler for button clicks
send_button.on_click(on_send_button_clicked)
map_button.on_click(on_map_button_clicked)

# Display the UI components
display(widgets.VBox([input_box, send_button, map_button, output_area]))


In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains import ConversationChain
from langchain.prompts import PromptTemplate
import folium
import warnings

# Suppress specific LangChain deprecation warnings
warnings.filterwarnings("ignore", category=UserWarning, message="LangChainDeprecationWarning")

# Set up memory to hold a rolling window of the last 100 exchanges
memory = ConversationBufferWindowMemory(k=100)

# Define the prompt template with both `history` and `input` variables
PROMPT_GUIDELINES = PromptTemplate(
    input_variables=["history", "input"],
    template="""
    You are a helpful assistant. Your name is Emily.
    Greet the user and briefly introduce yourself as their trip planner.
    Ask for the kind of tour they want and gather preferences before suggesting a route.
    Answer based on {history} and {input}
    """
)

# Create the conversation chain with memory and prompt guidelines
conversation = ConversationChain(llm=llm, memory=memory, prompt=PROMPT_GUIDELINES)

# Initialize interface components
input_box = widgets.Text(placeholder='Enter your query here...')
output_area = widgets.Output()
send_button = widgets.Button(description="Send")
map_output = widgets.Output()

# Initial greeting to introduce Ask Emily to the user
initial_greeting = """
Hello! I'm Ask Emily, your AI-powered Trip Planner, Travel Advisor, GPS, and RV Technician Finder.
I can help plan your journey, find RV-safe routes, and locate nearby campgrounds or technicians.
Let me know how I can assist you with your trip!
"""

# Display the initial greeting
with output_area:
    clear_output()
    display(widgets.HTML(f"<b>Ask Emily:</b> {initial_greeting}"))

# Function to create and display the map
def create_map(route_coords):
    # Create a folium map centered around the first coordinate
    my_map = folium.Map(location=route_coords[0], zoom_start=8)

    # Add markers for each point in the route
    for coord in route_coords:
        folium.Marker(location=coord).add_to(my_map)

    # Draw the route
    folium.PolyLine(locations=route_coords, color='blue', weight=2.5, opacity=1).add_to(my_map)

    return my_map

# Function to handle conversation with the LLM
def on_send_button_clicked(b):
    user_input = input_box.value
    input_box.value = ""  # Clear the input box after sending

    if user_input.strip():  # Process only if input is not empty
        # Display user's input in conversation history
        with output_area:
            display(widgets.HTML(f"<b>User:</b> {user_input}"))

        # Get the LLM response based on memory and prompt
        response = conversation.predict(input=user_input)

        # Display the LLM response in conversation history
        with output_area:
            display(widgets.HTML(f"<b>Ask Emily:</b> {response}"))

        # Extract route coordinates from the response (dummy data for demonstration)
        # In your implementation, extract actual coordinates from LLM response
        route_coords = [(25.4945, 91.9789), (25.4965, 91.9889), (25.4985, 91.9999)]  # Example coordinates

        # Create and display the map
        with map_output:
            clear_output()
            map_display = create_map(route_coords)
            display(map_display)

# Event handler for button click
send_button.on_click(on_send_button_clicked)

# Display the UI components
display(widgets.VBox([input_box, send_button, output_area, map_output]))


In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains import ConversationChain
from langchain.prompts import PromptTemplate
import warnings

# Suppress specific LangChain deprecation warnings
warnings.filterwarnings("ignore", category=UserWarning, message="LangChainDeprecationWarning")

# Set up memory to hold a rolling window of the last 100 exchanges
memory = ConversationBufferWindowMemory(k=100)

# List of questions to ask the user
questions = [
    "What type of tour are you interested in (e.g., historical places, temples, waterfalls, rivers, oceans)?",
    "What is your preferred mode of transportation (e.g., bike, car, public transport)?",
    "Do you have any specific destinations in mind?",
    "How many days do you plan to travel?",
    "What are your interests during the trip (e.g., adventure, relaxation, sightseeing)?"
]

# Define the prompt template for asking questions
ASK_PROMPT = PromptTemplate(
    input_variables=["history", "input"],
    template="""
    You are a helpful assistant named Emily.
    Please ask the user the following question based on their input history:

    {input}

    Previous conversation history: {history}
    """
)

# Define the prompt template for generating recommendations
RECOMMENDATION_PROMPT = PromptTemplate(
    input_variables=["history"],
    template="""
    Based on the user's preferences and previous conversation history, please provide a structured bullet-point list of recommendations.

    Previous conversation history: {history}
    """
)

# Create the conversation chain with memory and prompt guidelines
conversation = ConversationChain(llm=llm, memory=memory, prompt=ASK_PROMPT)

# Initialize interface components
input_box = widgets.Text(placeholder='Enter your query here...')
output_area = widgets.Output()
send_button = widgets.Button(description="Send")

# Initial greeting to introduce Ask Emily to the user
initial_greeting = """
Hello! I'm Ask Emily, your AI-powered Trip Planner, Travel Advisor, GPS, and RV Technician Finder.
I can help plan your journey, find RV-safe routes, and locate nearby campgrounds or technicians.
Let me know how I can assist you with your trip!
"""

# Display the initial greeting
with output_area:
    clear_output()
    display(widgets.HTML(f"<b>Ask Emily:</b> {initial_greeting}"))

# Function to handle conversation with the LLM
def on_send_button_clicked(b):
    user_input = input_box.value
    input_box.value = ""  # Clear the input box after sending

    if user_input.strip():  # Process only if input is not empty
        # Display user's input in conversation history
        with output_area:
            display(widgets.HTML(f"<b>User:</b> {user_input}"))

        # Check if we are still in the question-asking phase
        if len(memory.buffer) < len(questions):
            # Get the next question to ask
            next_question = questions[len(memory.buffer)]
            response = conversation.predict(input=next_question)  # Ask the next question
        else:
            # Move to recommendations after all questions have been asked
            response = conversation.predict(input="Let's summarize your preferences and provide recommendations.")
            # Generate recommendations based on history
            recommendations = conversation.predict(input=RECOMMENDATION_PROMPT.format(history=memory.buffer))
            response += "\nHere are some recommendations based on your preferences:\n" + recommendations

        # Display the LLM response in conversation history
        with output_area:
            display(widgets.HTML(f"<b>Ask Emily:</b> {response}"))

# Event handler for button click
send_button.on_click(on_send_button_clicked)

# Display the UI components
display(widgets.VBox([input_box, send_button, output_area]))


In [ ]:
# === Chat Interface Setup ===

import ipywidgets as widgets
from IPython.display import display, clear_output

# Initialize interface components
input_box = widgets.Text(placeholder='Enter your query here...')
output_area = widgets.Output()
send_button = widgets.Button(description="Send")

# Initial greeting to introduce Ask Emily to the user
initial_greeting = """
Hello! I'm Ask Emily, your AI-powered Trip Planner, Travel Advisor, GPS, and RV Technician Finder.
I can help plan your journey, find RV-safe routes, and locate nearby campgrounds or technicians.
Let me know how I can assist you with your trip!
"""
with output_area:
    display(widgets.HTML(f"<b>Ask Emily:</b> {initial_greeting}"))

# Define prompt guidelines for concise, goal-focused responses
PROMPT_GUIDELINES = """
Please keep your responses compact and relevant. Respond only based on the following topics:
- Trip planning based on RV specifications.
- Suggested routes for specific terrains (e.g., hills, beaches).
- Finding nearby campgrounds, RV parks, or fuel stations.
- Locating RV technicians and repair services.
- Informing users about route constraints (e.g., low bridges, diesel availability).
"""

# Function to handle conversation with the LLM
def on_send_button_clicked(b):
    user_input = input_box.value
    input_box.value = ""  # Clear the input box after sending

    with output_area:
        # Display user input
        display(widgets.HTML(f"<b>User:</b> {user_input}"))
        clear_output(wait=True)

        # Send the user input as a prompt to the LLM
        response = llm.invoke(f"{PROMPT_GUIDELINES}\nUser input: {user_input}")  # Use LLM to get the response

        # Display the response from the LLM
        display(widgets.HTML(f"<b>Ask Emily:</b> {response}"))

# Event handler for button click
send_button.on_click(on_send_button_clicked)

# Display the UI components
display(widgets.VBox([input_box, send_button, output_area]))


In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# Initialize interface components
input_box = widgets.Text(placeholder='Enter your query here...')
output_area = widgets.Output()
send_button = widgets.Button(description="Send")

# Initialize conversation buffer to retain context
conversation_buffer = []

# Initial greeting to introduce Ask Emily to the user
initial_greeting = """
Hello! I'm Ask Emily, your AI-powered Trip Planner, Travel Advisor, GPS, and RV Technician Finder.
I can help plan your journey, find RV-safe routes, and locate nearby campgrounds or technicians.
Let me know how I can assist you with your trip!
"""
conversation_buffer.append(f"Ask Emily: {initial_greeting}")

# Display initial greeting
with output_area:
    display(widgets.HTML(f"<b>Ask Emily:</b> {initial_greeting}"))

# Define prompt guidelines for concise, goal-focused responses
PROMPT_GUIDELINES = """
Please keep your responses compact and relevant. Respond only based on the following topics:
- Trip planning based on RV specifications.
- Suggested routes for specific terrains (e.g., hills, beaches).
- Finding nearby campgrounds, RV parks, or fuel stations.
- Locating RV technicians and repair services.
- Informing users about route constraints (e.g., low bridges, diesel availability).
"""

# Function to handle conversation with the LLM
def on_send_button_clicked(b):
    user_input = input_box.value
    input_box.value = ""  # Clear the input box after sending

    # Add user input to conversation buffer
    conversation_buffer.append(f"User: {user_input}")

    # Prepare prompt with buffer memory
    conversation_history = "\n".join(conversation_buffer[-6:])  # Limit memory to last 6 interactions
    prompt = f"{PROMPT_GUIDELINES}\n\nConversation history:\n{conversation_history}\n\nUser input: {user_input}"

    with output_area:
        # Display user input
        display(widgets.HTML(f"<b>User:</b> {user_input}"))
        clear_output(wait=True)

        # Send the prompt to the LLM for response
        response = llm.invoke(prompt)  # Use LLM to get the response

        # Display response from LLM
        display(widgets.HTML(f"<b>Ask Emily:</b> {response}"))

        # Add LLM response to conversation buffer
        conversation_buffer.append(f"Ask Emily: {response}")

# Event handler for button click
send_button.on_click(on_send_button_clicked)

# Display the UI components
display(widgets.VBox([input_box, send_button, output_area]))


In [ ]:
%pip install googlemaps

  Preparing metadata (setup.py) ... done
  Created wheel for googlemaps: filename=googlemaps-4.10.0-py3-none-any.whl size=40715 sha256=d743f73c0ff30b40b9aa7e4b2af7d196aaf6e90328b78f09d1225160fee52dc7
  Stored in directory: /root/.cache/pip/wheels/17/f8/79/999d5d37118fd35d7219ef57933eb9d09886c4c4503a800f84
Successfully built googlemaps


In [ ]:
import googlemaps

# Initialize the client
gmaps = googlemaps.Client(key='GOOGLE_MAPS_API_KEY')

# Define function to get route with constraints
def get_route_with_constraints(origin, destination, vehicle_height, vehicle_weight):
    route = gmaps.directions(origin, destination, mode="driving",
                             avoid="ferries",  # Example constraint
                             region="us")
    # Filter based on vehicle specs (height/weight)
    filtered_route = [leg for leg in route if validate_route(leg, vehicle_height, vehicle_weight)]
    return filtered_route

def validate_route(leg, height, weight):
    # Placeholder for detailed route validation based on clearance data
    return True  # Simulate filtering based on constraints


ValueError: Invalid API key provided.